In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [6]:
# Reading the file
with open("./brf_2.yml", 'r') as f:
    params = yaml.safe_load(f)
input_data_path = params['input_data_path']
df = pd.read_csv(input_data_path)
df.head(3)

In [42]:
df.shape

(132, 2460)

In [14]:
#Create train and test datasets
train = df.iloc[0:100,:]
test = df.iloc[100:,:]

x_test= test.drop(['Close^IXIC'],axis=1)
y_test = test['Close^IXIC']
x_train= train.drop(['Close^IXIC'],axis=1)
y_train = train['Close^IXIC']

In [15]:
def LR(x_train,y_train,y_test,test_features):
    """
    Function to calculate MAPE of a list of features using Linear regression
    
    # Arguments
        x_train: Train dataframe (independednt features)
        y_train: Train dataframe (dependednt feature)
        y_test: Test dataframe (dependednt feature)
        test_features: List of features
        
    # Returns
        List: MAPE on the list of features
    """

    mape_l=[] # List to hold the MAPE score
    lr=LinearRegression()
    
    for predictors in test_features:
        lr.fit(x_train.loc[:,predictors],y_train)
        predict= lr.predict(x_test.loc[:,predictors])
        mape = np.mean(abs(predict - y_test)/y_test)
        mape_l.append(mape)
    return(mape_l)  

In [17]:
%%time
features=list(x_test.columns)
count=1
test_features = [] 
best_features = []
while(count<=8):
    for i in features:
        test_features.append(best_features + [i])
    result = LR(x_train,y_train,y_test,test_features)
    best_features = test_features[np.argmin(result)]
    print('best_features:',best_features)
    features.remove(best_features[-1])
    test_features = []
    count = count + 1

best_features: ['Open^IXIC']
best_features: ['Open^IXIC', 'Low^IXIC']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC', 'High2086.HK']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC', 'High2086.HK', 'CloseKSE.BO']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC', 'High2086.HK', 'CloseKSE.BO', 'LowLUS.SG']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC', 'High2086.HK', 'CloseKSE.BO', 'LowLUS.SG', 'Close8340.HK']
best_features: ['Open^IXIC', 'Low^IXIC', 'High^IXIC', 'High2086.HK', 'CloseKSE.BO', 'LowLUS.SG', 'Close8340.HK', 'LowS7N1.MU']


In [20]:
x_test_imp = x_test.loc[:,best_features]
x_train_imp = x_train.loc[:,best_features]

In [24]:
lr=LinearRegression()
lr.fit(x_train.loc[:,best_features],y_train)
predict= lr.predict(x_test.loc[:,best_features])
mape = np.mean(abs(predict - y_test)/y_test)
mape

0.00043918480253505384

### RF Implementation

In [38]:
def RF(x_train,y_train,y_test,test_features):
    """
    Function to calculate MAPE of a list of features using Random Forest
    
    # Arguments
        x_train: Train dataframe (independednt features)
        y_train: Train dataframe (dependednt feature)
        y_test: Test dataframe (dependednt feature)
        test_features: List of features
        
    # Returns
        List: MAPE on the list of features
    """
    mape_l=[]

    for predictors in test_features:
        rf=RandomForestRegressor()
        rf.fit(x_train.loc[:,predictors],y_train)
        predict= rf.predict(x_test.loc[:,predictors])
        mape = np.mean(abs(predict - y_test)/y_test)
        mape_l.append(mape)
    return(mape_l)

In [40]:
%%time
features=list(x_test.columns)
count=1
test_features = [] 
best_features = []
while(count<=8):
    for i in features:
        test_features.append(best_features + [i])
    result = RF(x_train,y_train,y_test,test_features)
    best_features = test_features[np.argmin(result)]
    print('best_features:',best_features)
    features.remove(best_features[-1])
    test_features = []
    count = count + 1

best_features: ['Low^IXIC']
best_features: ['Low^IXIC', 'HighMEOAF']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ', 'CloseMEOAF']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ', 'CloseMEOAF', 'Close0090.HK']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ', 'CloseMEOAF', 'Close0090.HK', 'CloseLA5.SG']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ', 'CloseMEOAF', 'Close0090.HK', 'CloseLA5.SG', 'Close033340.KQ']
best_features: ['Low^IXIC', 'HighMEOAF', 'Low093520.KQ', 'CloseMEOAF', 'Close0090.HK', 'CloseLA5.SG', 'Close033340.KQ', 'Low033340.KQ']


In [ ]:
x_test_imp = x_test.loc[:,best_features]
x_train_imp = x_train.loc[:,best_features]

In [41]:
rf=RandomForestRegressor()
rf.fit(x_train.loc[:,best_features],y_train)
predict= rf.predict(x_test.loc[:,best_features])
mape = np.mean(abs(predict - y_test)/y_test)
mape

0.09555030125904429